## Основные задачи для работы с различными расширениями файлов
### (выполнил Валяев Георгий Анатольевич, студент группы ПИ20-6)

Датасеты:
* addres-book.json
* addres-book.xml
* addres-book-q.xml

In [1]:
import json, requests
from bs4 import BeautifulSoup
import pandas as pd
import pickle, os, shelve, time
import numpy as np
import xml.etree.ElementTree as xml

In [21]:
# библиотека для нормальной работы pandas с excel-файлами
!pip3 install lxml

You should consider upgrading via the 'C:\Users\admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


# JSON

* Работаем с файлом addres-book.json

In [2]:
with open('addres-book.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)
data

[{'name': 'Faina Lee',
  'email': 'faina@mail.ru',
  'birthday': '22.08.1994',
  'phones': [{'phone': '232-19-55'}, {'phone': '+7 (916) 232-19-55'}]},
 {'name': 'Robert Lee',
  'email': 'robert@mail.ru',
  'birthday': '22.08.1994',
  'phones': [{'phone': '111-19-55'}, {'phone': '+7 (916) 445-19-55'}]}]

1) Вывести все адреса электронной почты, содржащиеся в адресной книге, имеющей структуру, аналогичную my_ab2.

In [3]:
print(*[i['email'] for i in data], sep='\n')

faina@mail.ru
robert@mail.ru


2) Вывести телефоны, содржащиеся в адресной книге, имеющей структуру, аналогичную my_ab2.

In [4]:
print(*[[j['phone'] for j in i['phones']] for i in data], sep='\n')

['232-19-55', '+7 (916) 232-19-55']
['111-19-55', '+7 (916) 445-19-55']


3) Изменить структуру данных my_ab2 так, чтобы для телефонов был однозначно известен их тип. Создать пример записи с обновленной структурой данных. Вывести все домашние телефоны, содржащиеся в созданном примере.

In [5]:
for elem in data:
    for number_phone in elem['phones']:
        if number_phone['phone'][0] == '+':
            elem['work_number_phone'] = number_phone['phone']
        else:
            elem['home_number_phone'] = number_phone['phone']
    elem.pop('phones')
data

[{'name': 'Faina Lee',
  'email': 'faina@mail.ru',
  'birthday': '22.08.1994',
  'home_number_phone': '232-19-55',
  'work_number_phone': '+7 (916) 232-19-55'},
 {'name': 'Robert Lee',
  'email': 'robert@mail.ru',
  'birthday': '22.08.1994',
  'home_number_phone': '111-19-55',
  'work_number_phone': '+7 (916) 445-19-55'}]

3.2) Изменить структуру данных my_ab2 так, чтобы для каждой записи в адресной книге можно было записать список друзей этого человека, присутствующих в этой же адресной книге.

In [6]:
for user in data:
    user['friends'] = []
data

[{'name': 'Faina Lee',
  'email': 'faina@mail.ru',
  'birthday': '22.08.1994',
  'home_number_phone': '232-19-55',
  'work_number_phone': '+7 (916) 232-19-55',
  'friends': []},
 {'name': 'Robert Lee',
  'email': 'robert@mail.ru',
  'birthday': '22.08.1994',
  'home_number_phone': '111-19-55',
  'work_number_phone': '+7 (916) 445-19-55',
  'friends': []}]

4) Написать функцию, которая формирует новую запись в телефонной книге, запрашивая у пользователя поочередно все необходимые поля.

In [7]:
def write_user(data):
    user_dict = {
        'name': '', 'email': '', 'birthday': ''
    }
    name = input("Введите имя: ")
    email = input("Введите электронную почту: ")
    birthday = input("Введите дату рождения в формате YY.MM.DD: ")
    is_right = True
    while is_right:
        phone = input('Какой из типов номеров желаете ввести: рабочий (work) или домашний (home)?')
        if (phone == 'work') or (phone == 'рабочий'):
            number_phone = input('Введите номер телефона по такому шаблону => +7 (...) ...-..-.. => ')
            user_dict['work_number_phone'] = number_phone
            continue_point = input('Желаете добавить дополнительный номер телефона? да или нет? => ')
            if continue_point.lower() != 'да':
                break
        elif (phone == 'home') or (phone == 'домашний'):
            number_phone = input('Введите номер телефона по такому шаблону => ...-..-.. => ')
            user_dict['home_number_phone'] = number_phone
            continue_point = input('Желаете добавить дополнительный номер телефона? да или нет? => ')
            if continue_point.lower() != 'да':
                break
    lst_fr = input('Желаете ввести своих друзей в книжку? да или нет? => ')
    if lst_fr.lower() == 'да':
        friends = input('Введите имена ваших друзей через запятую => ').split(',')
    user_dict['name'], user_dict['email'], user_dict['birthday'], user_dict['friends'] = name, email, birthday, friends
    data.append(user_dict)
    return data

In [8]:
write_user(data)

[{'name': 'Faina Lee',
  'email': 'faina@mail.ru',
  'birthday': '22.08.1994',
  'home_number_phone': '232-19-55',
  'work_number_phone': '+7 (916) 232-19-55',
  'friends': []},
 {'name': 'Robert Lee',
  'email': 'robert@mail.ru',
  'birthday': '22.08.1994',
  'home_number_phone': '111-19-55',
  'work_number_phone': '+7 (916) 445-19-55',
  'friends': []},
 {'name': 'Саша',
  'email': 'shasa2084@gmail.com',
  'birthday': '12.03.1999',
  'home_number_phone': '549-31-43',
  'friends': ['Михаил', 'Олег', 'Кирилл', 'Егор', 'Владимир', 'Алексей']}]

5) Написать функцию, которая ищет по имени в хранящейся в файле телефонной книге запись о человеке и выводит информацию о нем в виде текста, где в каждой строке указывается тип выводимого поля и его содержимое.

In [9]:
def search_name(name, data):
    for i in data:
        if i['name'].lower() == name.lower():
            for j in i:
                try:
                    if i[j] != '':
                        print(j + ': ' + i[j])
                    else:
                        print(j + ": нет данных")
                except TypeError:
                    print(j + ': ' + ', '.join(i[j]))
search_name('Саша', data)

name: Саша
email: shasa2084@gmail.com
birthday: 12.03.1999
home_number_phone: 549-31-43
friends: Михаил, Олег, Кирилл, Егор, Владимир, Алексей


6) Получить координаты всех отделений связи г. Пущино, имеющих отдельный почтовый индекс, с использованием API 'http://api.zippopotam.us' . (Индексы использыемые в г. Пущино нужно собрать из открытых источников).

In [10]:
postcode = '142290'
url = 'https://api.zippopotam.us/ru/' + postcode
resp = requests.get(url=url).json()
resp

{'post code': '142290',
 'country': 'Russia',
 'country abbreviation': 'RU',
 'places': [{'place name': 'Пущино',
   'longitude': '37.349',
   'state': 'Московская Область',
   'state abbreviation': '',
   'latitude': '54.9454'}]}

In [11]:
print('Координаты отделения связи г. Пущино:', *[{'Долгота': pl['longitude'], 'Широта': pl['latitude']} for pl in resp['places']], sep='\n')

Координаты отделения связи г. Пущино:
{'Долгота': '37.349', 'Широта': '54.9454'}


# Pickle

1) Сгенерировать данные, аналогичные структуре addres-book.json с 1000 записей.

In [12]:
gener_dataframe = pd.DataFrame()

gener_dataframe['name'] = [f'Name №{i}' for i in range(1000)]
gener_dataframe['email'] = [f"201174_{i}@gmail.com" for i in range(1000)]
gener_dataframe['birthday'] = [f'08.08.{1022 + i}' for i in range(1000)]
gener_dataframe['home_number_phone'] = [f'777-77-77' for _ in range(1000)]

gener_dataframe

,name,email,birthday,home_number_phone
0,Name №0,201174_0@gmail.com,08.08.1022,777-77-77
1,Name №1,201174_1@gmail.com,08.08.1023,777-77-77
2,Name №2,201174_2@gmail.com,08.08.1024,777-77-77
3,Name №3,201174_3@gmail.com,08.08.1025,777-77-77
4,Name №4,201174_4@gmail.com,08.08.1026,777-77-77
...,...,...,...,...
995,Name №995,201174_995@gmail.com,08.08.2017,777-77-77
996,Name №996,201174_996@gmail.com,08.08.2018,777-77-77
997,Name №997,201174_997@gmail.com,08.08.2019,777-77-77
998,Name №998,201174_998@gmail.com,08.08.2020,777-77-77


2) Сохранить результаты з.1 в файл Pickle и в файл JSON. Сравнить объемы получившихся файлов.
3) Сравнить время записи / чтения данных в форматах Pickle и JSON из з.2

In [13]:
def save_pickle(df):
    pickle.dump(df, open("data/new-address-book.bin", "wb"))

def save_json(df):
    df.to_json("data/new-address-book.json")

def read_pickle():
    f = open('data/new-address-book.bin', 'rb')
    res = pickle.load(f)
    f.close()
    return res

def read_json():
    f = open('data/new-address-book.json', 'r')
    res = json.load(f)
    f.close()
    return res

##### Время записи данных в файлы pickle & json

In [14]:
%timeit save_pickle(gener_dataframe)

1.11 ms ± 204 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [15]:
%timeit save_json(gener_dataframe)

3.06 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


##### Сравнение объёмов получившихся файлов

In [16]:
pickle_file_stats, json_file_stats = os.stat('data/new-address-book.bin'), os.stat('data/new-address-book.json')
print(f'Pickle-файл: {pickle_file_stats.st_size} б\nJson-файл: {json_file_stats.st_size} б')
print("Pickle-файл легче по размеру json в ", end='')
print(round(json_file_stats.st_size / pickle_file_stats.st_size, 2), 'раз(-а)')

Pickle-файл: 55629 б
Json-файл: 88395 б
Pickle-файл легче по размеру json в 1.59 раз(-а)


##### Время чтения файлов pickle & json

In [19]:
%timeit read_pickle()

937 µs ± 259 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [18]:
%timeit read_json()

1.38 ms ± 176 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [20]:
pickle_t = []
for i in range(10000):
    time_start = time.time()
    pickle.load(open("data/new-address-book.bin", "rb"))
    total_time = time.time() - time_start
    pickle_t.append(total_time)

print(f'Чтение файла pickle в среднем занимает {np.mean(pickle_t)} мс.')
pickle_t_mean = np.mean(pickle_t)

Чтение файла pickle в среднем занимает 0.0010131719827651978 мс.


In [21]:
json_t = []
for i in range(1000):
    time_start = time.time()
    json.load(open('data/new-address-book.json', 'r'))
    total_time = time.time() - time_start
    json_t.append(total_time)

print(f'Чтение файла json занимает в среднем {np.mean(json_t)} мс.')
json_times_mean = np.mean(json_t)
print(f"Pickle в среднем быстрее JSON в: {round(json_times_mean / pickle_t_mean, 4)} раз.")

Чтение файла json занимает в среднем 0.002719596862792969 мс.
Pickle в среднем быстрее JSON в: 2.6842 раз.


4) Преобразовать структуру данных из addres-book.json так, чтобы для получения данных о конкретном человеке можно было эффективно использовать возможности модуля shelve.

In [22]:
data

[{'name': 'Faina Lee',
  'email': 'faina@mail.ru',
  'birthday': '22.08.1994',
  'home_number_phone': '232-19-55',
  'work_number_phone': '+7 (916) 232-19-55',
  'friends': []},
 {'name': 'Robert Lee',
  'email': 'robert@mail.ru',
  'birthday': '22.08.1994',
  'home_number_phone': '111-19-55',
  'work_number_phone': '+7 (916) 445-19-55',
  'friends': []},
 {'name': 'Саша',
  'email': 'shasa2084@gmail.com',
  'birthday': '12.03.1999',
  'home_number_phone': '549-31-43',
  'friends': ['Михаил', 'Олег', 'Кирилл', 'Егор', 'Владимир', 'Алексей']}]

In [23]:
json_data = pd.read_json("addres-book.json")

json_data["home_phone"] = [num[0]['phone'] for num in json_data['phones']]
json_data["work_phone"] = [num[1]['phone'] for num in json_data['phones']]

json_data = json_data.drop("phones", axis=1)

json_data

,name,email,birthday,home_phone,work_phone
0,Faina Lee,faina@mail.ru,22.08.1994,232-19-55,+7 (916) 232-19-55
1,Robert Lee,robert@mail.ru,22.08.1994,111-19-55,+7 (916) 445-19-55


In [24]:
shelf = shelve.open('shelve-db')
for user in json_data['name']:
    shelf[user] = json_data[json_data['name'] == user]

print(shelf['Faina Lee'])
shelf.close()

        name          email    birthday home_phone          work_phone
0  Faina Lee  faina@mail.ru  22.08.1994  232-19-55  +7 (916) 232-19-55


In [25]:
shelf = shelve.open('shelve-db')
print(shelf['Robert Lee'])
shelf.close()

         name           email    birthday home_phone          work_phone
1  Robert Lee  robert@mail.ru  22.08.1994  111-19-55  +7 (916) 445-19-55


In [26]:
with shelve.open("shelve-db") as shelve_file:
    for item in shelve_file.items():
        print(item)

('Faina Lee',         name          email    birthday home_phone          work_phone
0  Faina Lee  faina@mail.ru  22.08.1994  232-19-55  +7 (916) 232-19-55)
('Robert Lee',          name           email    birthday home_phone          work_phone
1  Robert Lee  robert@mail.ru  22.08.1994  111-19-55  +7 (916) 445-19-55)


# XML

1) По данным из файла addres-book-q.xml сформировать список словарей с телефоноами каждого из людей. Вид результата: [{ФИО: ['телефон1', 'телефон2']}] и вывести его на экран и сохранить в json файле.

In [27]:
print("Общий парсинг XML-файла ('addres-book-q.xml'):")
with open('addres-book-q.xml', 'r') as f:
    xml_data = BeautifulSoup(f, 'xml')

addrs_dict = {}
for address in xml_data.address_book.find_all('address'):
    data_lst = address.getText('|',strip=True).split('|')
    index = 'id_' + address['id']
    addrs_dict[index] = {}
    addrs_dict[index]['gender'] = data_lst[0]
    addrs_dict[index]['name'] = data_lst[1]
    addrs_dict[index]['email'] = data_lst[2]
    addrs_dict[index]['position'] = data_lst[3]
    addrs_dict[index]['company'] = data_lst[4]
    for phone in address.phones:
        if phone != '\n':
            addrs_dict[index][phone['type']] = phone.text
addrs_dict

Общий парсинг XML-файла ('addres-book-q.xml'):


{'id_1': {'gender': 'm',
  'name': 'Aicha Barki',
  'email': 'aiqraa.asso@caramail.com',
  'position': 'Presidente',
  'company': "Association Algerienne d'Alphabetisation Iqraa",
  'work': '+ (213) 6150 4015',
  'personal': '+ (213) 2173 5247'},
 'id_2': {'gender': 'm',
  'name': 'Francisco Domingos',
  'email': 'frandomingos@hotmail.com',
  'position': 'Directeur General',
  'company': 'Institut National de Education des Adultes',
  'work': '+ (244-2) 325 023',
  'personal': '+ (244-2) 325 023'},
 'id_3': {'gender': 'f',
  'name': 'Maria Luisa',
  'email': 'luisagrilo@ebonet.net',
  'position': 'Directrice Nationale',
  'company': 'Institut National de Education des Adultes',
  'personal': '+ (244) 4232 2836'},
 'id_4': {'gender': 'm',
  'name': 'Abraao Chanda',
  'email': 'ineda@snet.co.ao',
  'position': 'Chef',
  'company': 'Institut National de Education des Adultes',
  'work': '+ (244-2) 325 023',
  'personal': '+ (244-2) 325 023'},
 'id_5': {'gender': 'm',
  'name': 'Beatriz Bu

In [28]:
final_lst_numbers = []
for id_people in addrs_dict:
    people = {}
    people['ФИО'] = addrs_dict[id_people]['name']
    lst_keys = list(addrs_dict[id_people].keys())[5:]
    numbers = [addrs_dict[id_people][number] for number in lst_keys]
    if len(numbers) == 2:
        people['телефон_1'] = numbers[0]
        people['телефон_2'] = numbers[1]
    else:
        people['телефон_1'] = numbers[0]
    final_lst_numbers.append(people)
final_lst_numbers

[{'ФИО': 'Aicha Barki',
  'телефон_1': '+ (213) 6150 4015',
  'телефон_2': '+ (213) 2173 5247'},
 {'ФИО': 'Francisco Domingos',
  'телефон_1': '+ (244-2) 325 023',
  'телефон_2': '+ (244-2) 325 023'},
 {'ФИО': 'Maria Luisa', 'телефон_1': '+ (244) 4232 2836'},
 {'ФИО': 'Abraao Chanda',
  'телефон_1': '+ (244-2) 325 023',
  'телефон_2': '+ (244-2) 325 023'},
 {'ФИО': 'Beatriz Busaniche', 'телефон_1': '+ (54-11) 4784 1159'},
 {'ФИО': 'Francesca Beddie',
  'телефон_1': '+ (61-2) 6274 9500',
  'телефон_2': '+ (61-2) 6274 9513'},
 {'ФИО': 'Graham John Smith', 'телефон_1': '+ (61-3) 9807 4702'}]

In [30]:
last_lst = []
for members in final_lst_numbers:
    mem, keys = {}, list(members.keys())
    mem[members[keys[0]]] = list(set(members[elem] for elem in keys[1:]))
    last_lst.append(mem)

with open('number-data.json', 'w') as file:
    json.dump(last_lst, file)

print('Получившийся список словарей с номерами телефонов:')
print(*last_lst, sep="\n")

Получившийся список словарей с номерами телефонов:
{'Aicha Barki': ['+ (213) 6150 4015', '+ (213) 2173 5247']}
{'Francisco Domingos': ['+ (244-2) 325 023']}
{'Maria Luisa': ['+ (244) 4232 2836']}
{'Abraao Chanda': ['+ (244-2) 325 023']}
{'Beatriz Busaniche': ['+ (54-11) 4784 1159']}
{'Francesca Beddie': ['+ (61-2) 6274 9513', '+ (61-2) 6274 9500']}
{'Graham John Smith': ['+ (61-3) 9807 4702']}


2) По данным из файла addres-book-q.xml сформировать список (в виде списка в Python) адресов электронной почты всех женщин и вывести его на экран.

In [31]:
tree_book, emails = xml.parse('addres-book-q.xml'), []
root = tree_book.getroot()
for country in root:
    for address in country:
        if address.find("gender").text == 'f':
            email = address.find('email')
            emails.append(email.text)
print('Список электронных почт всех женщин:')
print(*emails, sep='\n')

Список электронных почт всех женщин:
luisagrilo@ebonet.net
f.beddie@ala.asn.au


3) По данным из файла addres-book-q.xml сформировать список (в виде списка в Python) всех участников с рабочими телефонами и вывести его на экран.

In [32]:
lst_id_people, work_num_lst = [], []
for country in root:
    for address in country:
        if 'work' in [phone.get("type") for phone in address.find("phones")]:
            lst_id_people.append(address.get("id"))

for p_id in lst_id_people:
    work_num_lst.append(root.find(f".//*[@id='{p_id}']").find("name").text)

print("Список участников в книге с рабочими номерами телефонов:")
print(*work_num_lst, sep='\n')

Список участников в книге с рабочими номерами телефонов:
Aicha Barki
Francisco Domingos
Abraao Chanda
Beatriz Busaniche
Francesca Beddie
Graham John Smith


4) По данным из файла addres-book-q.xml сформировать словарь вида {название страны: количество участников} и вывести его на экран.

In [33]:
print("Словарь количества участников с группировкой по странам:")
{country.get('name'): len(country) for country in root}

Словарь количества участников с группировкой по странам:


{'algeria': 1, 'angola': 3, 'argentina': 1, 'australia': 2}

5) Превратить информацию из addres-book.xml в json файл с содержимым вида:

[{'birthday': '11.07.1984', 'email': 'bruce@gmail.com', 'name': 'Bruce Lee'},
 {'birthday': '22.03.1985', 'email': 'alee@yandex.ru', 'name': 'Alice Lee'}]

In [34]:
xml_data = pd.read_xml('addres-book.xml')
xml_data.drop(columns=["phones", "id"], inplace=True)
lst_data, big_dict = [], {p: dict(xml_data[p]) for p in xml_data}

for i in range(2):
    human_data = {"birthday": big_dict['birthday'][i], "email": big_dict['email'][i], "name": big_dict['name'][i]}
    lst_data.append(human_data)

print("Список информации из json-файла:")
print(*lst_data, sep='\n')

Список информации из json-файла:
{'birthday': '11.07.1984', 'email': 'bruce@gmail.com', 'name': 'Bruce Lee'}
{'birthday': '22.03.1985', 'email': 'alee@yandex.ru', 'name': 'Alice Lee'}
